# Atlas of Genetics and Cytogenetics in Oncology and Haematology

In [1]:
import requests
from bs4 import BeautifulSoup
import re
from typing import NamedTuple
from collections import Counter
from urllib.parse import urljoin
import pickle

## Pull tables

In [2]:
chromosomes = [
    'X', 'Y'
] + list(range(1,23))
url = 'http://atlasgeneticsoncology.org/Indexbychrom/idxa_X.html'

In [3]:
# load responses
with open('data/atlas_responses.pkl', 'rb') as f:
    chr_resp_objects = pickle.load(f)

## Tokenize text

In [4]:
band_chars = r'[pq][0-9.]+'
chr_chars = r'[XY0-9]+'

iscn_translocation_re = r't' +\
    f'\((?P<chr1>{chr_chars});(?P<chr2>{chr_chars})\)' +\
    f'\((?P<band1>{band_chars});(?P<band2>{band_chars})\)'

gene_fusion_re = r'(?P<five_p_gene>\w+|\?)/(?P<three_p_gene>\w+|\?)'

band_only_re = f'\\((?P<bo_chr>{chr_chars})(?P<bo_band>{band_chars})\\)'

tissue_context_re = r'\(((Bone)|(Kidney)|(Soft Tissue(s)?)|(Lung)|(Breast)|(Prostate))( tumors)?\)'
tissue_context_re2 = r'(solely )?in [\w\s]+'

iscn_region_dup_re = r'dup' +\
    f'\((?P<dup_chr>{chr_chars})\)' +\
    f'\((?P<dup_region_start>{band_chars})-?(?P<dup_region_stop>{band_chars})\)'

iscn_region_del_re = r'del' +\
    f'\((?P<del_chr>{chr_chars})\)' +\
    f'\((?P<del_region_start>{band_chars})-?(?P<del_region_stop>{band_chars})\)'

chr_amp_re = r'\+' + chr_chars

token_specification = [
    ('TRANSLOCATION', iscn_translocation_re),
    ('GENE_FUSION', gene_fusion_re),
    ('REGION_DUP', iscn_region_dup_re),
    ('REGION_DEL', iscn_region_del_re),
    ('CHR_AMP', chr_amp_re),
    ('SKIP', r'[ \t]+'),
    ('BAND_ONLY', band_only_re),
    ('TISSUE_CONTEXT', tissue_context_re),
    ('TISSUE_CONTEXT_2', tissue_context_re2),
    ('FAIL', r'.')
]

class Token(NamedTuple):
    type: str
    value: str
    groupdict: dict

def tokenize(string):
    tokens = list()
    tok_regex = '|'.join('(?P<%s>%s)' % pair for pair in token_specification)
    
    for mo in re.finditer(tok_regex, string, flags=re.IGNORECASE):
        kind = mo.lastgroup
        value = mo.group()
        groupdict = {k:v for k, v in mo.groupdict().items() if v and k != kind}
        if kind == 'SKIP':
            continue
        elif kind == 'FAIL':
            raise RuntimeError(f'{value!r} unexpected.')
        tokens.append(Token(kind, value, groupdict))
    return tokens

test_str = 'del(X)(p22p22) P2RY8/CRLF2'
m = tokenize(test_str)
re.match(iscn_region_del_re, 'del(X)(p22p22)')
# re.compile(iscn_region_del_re).pattern
# tokenize('t(X;1)(p22;p34) PTPRF/RPS6KA3')

<re.Match object; span=(0, 14), match='del(X)(p22p22)'>

In [5]:
m

[Token(type='REGION_DEL', value='del(X)(p22p22)', groupdict={'del_chr': 'X', 'del_region_start': 'p22', 'del_region_stop': 'p22'}),
 Token(type='GENE_FUSION', value='P2RY8/CRLF2', groupdict={'five_p_gene': 'P2RY8', 'three_p_gene': 'CRLF2'})]

In [6]:
expected_headers = ['Annotated Leukemias', 'Other Leukemias', 'Annotated Tumors', 'Other Tumors']

In [7]:
c = Counter()
matches = list()
failures = list()

for chromosome, r in chr_resp_objects.items():
    s = BeautifulSoup(r.content, 'html.parser')
    tables = s.find_all('table', class_='sortable')
    for table in tables[:4]:
        header = table.th.text.strip()
        assert header in expected_headers
        for data in table.find_all('td'):
            href = data.a.attrs['href']
            text = data.a.text
            try:
                tokens = tokenize(text)
                k = ':'.join([header, 'match'])
                matches.append({
                    'input': text,
                    'tokens': tokens,
                    'url': urljoin(url, href),
                    'annot_type': header,
                })
            except RuntimeError as e:
                k = ':'.join([header, 'fail'])
                failures.append(text)
            c[k] += 1
match = len(matches)
failure = len(failures)
                
print(f'Succeeded on {match} of {match + failure} records ({match / (match + failure)})).')

Succeeded on 15068 of 15827 records (0.9520439754849308)).


In [8]:
matches[0]

{'input': 'del(X)(p22p22) P2RY8/CRLF2',
 'tokens': [Token(type='REGION_DEL', value='del(X)(p22p22)', groupdict={'del_chr': 'X', 'del_region_start': 'p22', 'del_region_stop': 'p22'}),
  Token(type='GENE_FUSION', value='P2RY8/CRLF2', groupdict={'five_p_gene': 'P2RY8', 'three_p_gene': 'CRLF2'})],
 'url': 'http://atlasgeneticsoncology.org/Anomalies/delXp22PR2Y8-CRLF2ID1599.html',
 'annot_type': 'Annotated Leukemias'}

In [9]:
total = sum(c.values())
fails = sum([v for k, v in c.items() if k.endswith('fail')])
successes = total - fails
for k in sorted(c):
    print(f'{k}: {c[k]}')
print(total)
print(f'{successes / total * 100:.1f}% success.')

Annotated Leukemias:fail: 308
Annotated Leukemias:match: 927
Annotated Tumors:fail: 210
Annotated Tumors:match: 608
Other Leukemias:fail: 104
Other Leukemias:match: 1233
Other Tumors:fail: 137
Other Tumors:match: 12300
15827
95.2% success.


## Construct locations and variants

### First, from Atlas dataset

In [10]:
from ga4gh.vr import models
from ga4gh.core import ga4gh_identify, ga4gh_serialize
import hashlib
import sys
import csv
from copy import copy

In [11]:
keys = [tuple([token.type for token in match['tokens']]) for match in matches]
Counter(keys)

Counter({('REGION_DEL', 'GENE_FUSION'): 65,
         ('CHR_AMP', 'TISSUE_CONTEXT_2'): 3,
         ('TRANSLOCATION', 'GENE_FUSION'): 10544,
         ('TRANSLOCATION',): 98,
         ('GENE_FUSION', 'BAND_ONLY'): 3686,
         ('TRANSLOCATION', 'GENE_FUSION', 'TISSUE_CONTEXT'): 603,
         ('REGION_DUP', 'TISSUE_CONTEXT_2'): 1,
         ('TRANSLOCATION', 'TISSUE_CONTEXT_2'): 11,
         ('REGION_DEL', 'GENE_FUSION', 'TISSUE_CONTEXT'): 3,
         ('REGION_DUP', 'GENE_FUSION'): 26,
         ('TRANSLOCATION', 'TISSUE_CONTEXT'): 2,
         ('TRANSLOCATION', 'GENE_FUSION', 'TISSUE_CONTEXT_2'): 26})

In [12]:
HGNC_PRIMARY = dict()
HGNC_ALIAS = dict()

with open('data/hgnc_table.tsv', 'r') as f:
    reader = csv.DictReader(f, delimiter='\t')
    for record in reader:
        v = record['HGNC ID'].lower()
        k = record['Approved symbol']
        HGNC_PRIMARY[k] = v
        
        previous_symbols = record['Previous symbols'].split(', ')
        alias_symbols = record['Alias symbols'].split(', ')
        for symbol in previous_symbols:
            HGNC_ALIAS[symbol.strip("'")] = v
        for symbol in alias_symbols:
            HGNC_ALIAS[symbol.strip("'")] = v


In [61]:
def vrs_hash(obj):
    digest_size = 24
    
    blob = ga4gh_serialize(obj)
    digest = hashlib.sha512(blob).digest()
    tdigest_int = int.from_bytes(digest[:digest_size], byteorder=sys.byteorder)
    return(tdigest_int)

hash_models = [
    models.ChromosomeLocation,
    models.GeneLocation,
    models.Allele,
    models.LocationJunction,
    models.RelativeAbundance,
    models.VariationSet
]

for m in hash_models:
    m.__hash__ = vrs_hash

def construct_VRS_chromosome_location(chromosome, start, end):
    interval = models.NamedInterval(start=start, end=end)
    location = models.ChromosomeLocation(
        species_id="taxonomy:9606",
        interval=interval,
        chr=chromosome
    )
    return location


def construct_VRS_gene_location(gene_symbol):
    if gene_symbol == '?':
        return None
    gene_id = HGNC_PRIMARY.get(gene_symbol, None)
    if gene_id is None:
        gene_id = HGNC_ALIAS.get(gene_symbol, None)
    if gene_id is None:
        gene_id=f"unidentified.symbol:{gene_symbol}"
    location = models.GeneLocation(
        gene_id=gene_id,
        interval=models.UndefinedInterval()
    )
    return location

def fusion_to_gene_locations(fusion_token):
    try:
        locations = [
            construct_VRS_gene_location( fusion_token.groupdict['five_p_gene'] ),
            construct_VRS_gene_location( fusion_token.groupdict['three_p_gene'] ),
        ]
    except KeyError:
        return []
    return locations
    
def del_to_region_location(del_token):
    l = construct_VRS_chromosome_location(
        chromosome=del_token.groupdict['del_chr'],
        start=del_token.groupdict['del_region_start'],
        end=del_token.groupdict['del_region_stop']
    )
    return l

def chr_amp_to_region_location(chr_amp_token):
    v = chr_amp_token.value
    interval = models.NamedInterval(start='pter', end='qter')
    location = models.ChromosomeLocation(
        species_id="taxonomy:9606",
        interval=interval,
        chr=v[1:]
    )
    return location
    
def translocation_to_region_locations(trx_token):
    locations = [(
        construct_VRS_chromosome_location(
            chromosome=trx_token.groupdict['chr1'],
            start='pter',
            end=trx_token.groupdict['band1']),
        construct_VRS_chromosome_location(
            chromosome=trx_token.groupdict['chr2'],
            start=trx_token.groupdict['band2'],
            end='qter')
        ),(
        construct_VRS_chromosome_location(
            chromosome=trx_token.groupdict['chr2'],
            start='pter',
            end=trx_token.groupdict['band2']),
        construct_VRS_chromosome_location(
            chromosome=trx_token.groupdict['chr1'],
            start=trx_token.groupdict['band1'],
            end='qter')
        )
    ]
    return locations

def dup_to_region_location(dup_token):
    location = construct_VRS_chromosome_location(
        chromosome=dup_token.groupdict['dup_chr'],
        start=dup_token.groupdict['dup_region_start'],
        end=dup_token.groupdict['dup_region_stop']
    )
    return location

def index_variant(variant, location, record, variant_index, record_index):
    if location is None:
        return
    l = variant_index.get(location, [])
    l.append(variant)
    variant_index[location] = l
    
    l = record_index.get(variant, [])
    l.append(record)
    record_index[variant] = l

In [62]:
atlas_variants = dict()
atlas_records = dict()

for match in matches:
    for token in match['tokens']:
        t = token.type
        if t == 'GENE_FUSION':
            genes = fusion_to_gene_locations(token)
            if not genes:
                continue
            variant = models.LocationJunction(
                left=genes[0],
                right=genes[1]
            )
            index_variant(variant, genes[0], match, atlas_variants, atlas_records)
            index_variant(variant, genes[1], match, atlas_variants, atlas_records)
        elif t == 'REGION_DEL':
            location = ( del_to_region_location(token) )
            state = models.ChromosomeState(
                molecularRegion=None
            )
            # Molecular Variant
            variant = models.Allele(
                location=location,
                state=state
            )
            index_variant(variant, location, match, atlas_variants, atlas_records)
            # Systemic Variant
            state = models.AmbiguousState()
            molecular_variation = models.Allele(
                location=location,
                state=state
            )
            variant = models.RelativeAbundance(
                molecularVariation=molecular_variation,
                relativeQuantity='less_than'
            )
            index_variant(variant, location, match, atlas_variants, atlas_records)
        elif t == 'REGION_DUP':
            location = ( dup_to_region_location(token) )
            # Note: dups (insertions at the original chromosome location) are awkward due to the use of regions as locations.
            #   e.g. how to differentiate insertion of Xq11-q44 at Xq11 with and without replacement of Xq11?
            #   per ISCN 2016 convention (Section 9.2.9) insertions occur AT a band, no further specificity.
            state = models.ChromosomeState(
                molecularRegion=location
            )
            insertion_location=construct_VRS_chromosome_location(location.chr, location.interval.end, location.interval.end)
            # Molecular Variant
            variant = models.Allele(
                location=insertion_location,
                state=state
            )
            index_variant(variant, location, match, atlas_variants, atlas_records)
            # Systemic Variant
            state = models.AmbiguousState()
            molecular_variation = models.Allele(
                location=location,
                state=state
            )
            variant = models.RelativeAbundance(
                molecularVariation=molecular_variation,
                relativeQuantity='greater_than'
            )
            index_variant(variant, location, match, atlas_variants, atlas_records)
        elif t == 'CHR_AMP':
            location = chr_amp_to_region_location(token)
            state=models.AmbiguousState()
            molecular_variation = models.Allele(
                location=location,
                state=state
            )
            variant = models.RelativeAbundance(
                molecularVariation=molecular_variation,
                relativeQuantity='greater_than'
            )
            index_variant(variant, location, match, atlas_variants, atlas_records)
        elif t == 'TRANSLOCATION':
            der1_regions, der2_regions = translocation_to_region_locations(token)
            # First derived chromosome
            der1 = models.LocationJunction(
                left=der1_regions[0],
                right=der1_regions[1]
            )
            # Second derived chromosome
            der2 = models.LocationJunction(
                left=der2_regions[0],
                right=der2_regions[1]
            )
            # Using a VariationSet in lieu of an explicit class for a Molecular Profile
            try:
                assert hash(der1) != hash(der2)
            except AssertionError:
                # Example value that triggers this is t(X;X)(p22;p22). Not clear what is meant, skip.
                continue
            variant = models.VariationSet(
                members=[der1, der2]
            )
            index_variant(variant, location, match, atlas_variants, atlas_records)
        elif 'CONTEXT' in t:
            continue # ignore context for now
        elif t == 'BAND_ONLY':
            continue # ignore band only as a variant
        else:
            raise NotImplementedError(f'Token type {t} not handled.')

In [63]:
len(atlas_variants)

8488

In [64]:
len(atlas_records)

14213

### TODO: EXPORT Annotations and VRS index files

Include `_meta` with license terms, dataset info, and VRS prototype disclaimer / info

## Second, from Project GENIE

v8.0 public, downloaded from SAGE: 10.7303/syn22228642

### Fusions

In [82]:
genie_fusion_samples = dict()
genie_sample_fusions = dict()
fusion_re = re.compile(r'(\w+)-(\w+) fusion')
failed = set()
evaluated = 0
skipped = 0
with open('data/genie_8_0_public/data_fusions.txt') as fusion_file:
    fusion_reader = csv.DictReader(fusion_file, delimiter='\t')
    for record in fusion_reader:
        evaluated += 1
        m = fusion_re.match(record['Fusion'])
        if m:
            left, right = map(construct_VRS_gene_location, m.groups())
            if not (left or right):
                skipped += 1
                continue
            variant = models.LocationJunction(
                left=left,
                right=right
            )
            s = genie_fusion_samples.get(variant, set())
            s.add(record['Tumor_Sample_Barcode'])
            genie_fusion_samples[variant] = s
            s = genie_sample_fusions.get(record['Tumor_Sample_Barcode'], set())
            s.add(variant)
            genie_sample_fusions[record['Tumor_Sample_Barcode']] = s
        else:
            skipped += 1

In [83]:
print(f'Intergenic, 2-partner fusions: {evaluated - skipped} ({(evaluated - skipped)/evaluated*100:.1f}% of records)')
print(f'Average fusions per sample: {sum(map(len, genie_sample_fusions.values())) / len(genie_sample_fusions):.2f}')

Intergenic, 2-partner fusions: 13686 (50.0% of records)
Average fusions per sample: 1.15


In [84]:
matched_fusions = dict()
for fusion in genie_fusion_samples:
    matches = set()
    if fusion.left:
        for variant in atlas_variants.get(fusion.left, []):
            if variant.type!=fusion.type:
                continue
            if variant.left != fusion.left:
                continue
            if variant.right is None or variant.right == fusion.right:
                matches.add(variant)
    if fusion.right:
        for variant in atlas_variants.get(fusion.right, []):
            if variant.type!=fusion.type:
                continue
            if variant.right != fusion.right:
                continue
            if variant.left is None or variant.left == fusion.left:
                matches.add(variant)
    if matches:
        matched_fusions[fusion] = matches

In [104]:
print(f'{len(matched_fusions)} of {len(genie_fusion_samples)} ({len(matched_fusions) / len(genie_fusion_samples) * 100:.1f}%) fusions with matches to Atlas.')
samples = len(genie_sample_fusions)
matched_samples = set()
for fusion in matched_fusions:
    matched_samples.update(genie_fusion_samples[fusion])
print(f'{len(matched_samples)} of {samples} ({len(matched_samples) / samples * 100:.1f}%) samples with fusion matches to Atlas.')

245 of 4241 (5.8%) fusions with matches to Atlas.
1186 of 5876 (20.2%) samples with fusion matches to Atlas.


In [100]:
matched_samples = set()
for fusion in matched_fusions:
    matched_samples.update(genie_fusion_samples[fusion])
samples

5876

### Questions
Which fusions are most frequently seen?

### Copy Number

For each gene-sample combination, a copy number level is specified:
- "-2" is a deep loss, possibly a homozygous deletion
- "-1" is a single-copy loss (heterozygous deletion)
- "0" is diploid
- "1" indicates a low-level gain
- "2" is a high-level amplification.